In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import pandas as pd
import time
import keyboard
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from userpasswd import username, passwd

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Define database and collection
db = client.soccer_db

In [3]:
#players = db.players_info.find()
player_info_df1 = pd.DataFrame(list(db.players_info.find()))
player_info_df1.head()

,_id,full_image_url,name,player_info
0,5c7de78a4537583a4cf57655,https://premierleague-static-files.s3.amazonaw...,Rolando Aarons,https://www.premierleague.com/players/10483/Ro...
1,5c7de78a4537583a4cf57656,https://premierleague-static-files.s3.amazonaw...,Tammy Abraham,https://www.premierleague.com/players/13286/Ta...
2,5c7de78a4537583a4cf57657,https://premierleague-static-files.s3.amazonaw...,Adrián,https://www.premierleague.com/players/4852/Adr...
3,5c7de78a4537583a4cf57658,https://premierleague-static-files.s3.amazonaw...,Adrien Silva,https://www.premierleague.com/players/11357/Ad...
4,5c7de78a4537583a4cf57659,https://premierleague-static-files.s3.amazonaw...,Benik Afobe,https://www.premierleague.com/players/4040/Ben...


In [4]:
player_info_df = player_info_df1[["name", "player_info", "full_image_url"]]
player_info_df.head()

,name,player_info,full_image_url
0,Rolando Aarons,https://www.premierleague.com/players/10483/Ro...,https://premierleague-static-files.s3.amazonaw...
1,Tammy Abraham,https://www.premierleague.com/players/13286/Ta...,https://premierleague-static-files.s3.amazonaw...
2,Adrián,https://www.premierleague.com/players/4852/Adr...,https://premierleague-static-files.s3.amazonaw...
3,Adrien Silva,https://www.premierleague.com/players/11357/Ad...,https://premierleague-static-files.s3.amazonaw...
4,Benik Afobe,https://www.premierleague.com/players/4040/Ben...,https://premierleague-static-files.s3.amazonaw...


In [5]:
con_string = f'{username}:{passwd}@127.0.0.1:3306/soccer_players'
conn_string = con_string
engine = create_engine(f'mysql://{conn_string}')
conn = engine.connect()

In [6]:
#player_data_df = pd.read_sql("select * from player_data where name like '%s''", names, engine)
player_data_df = pd.read_sql('select * from player_data', con=engine)
player_data_df.head()

,id,name,club,age,mkt_value,nationality
0,0,Alexis Sanchez,Arsenal,28,65.0,Chile
1,1,Mesut Ozil,Arsenal,28,50.0,Germany
2,2,Petr Cech,Arsenal,35,7.0,Czech Republic
3,3,Theo Walcott,Arsenal,28,20.0,England
4,4,Laurent Koscielny,Arsenal,31,22.0,France


In [7]:
#names = "abcdef"
names = "alex"
player_data_df["Select"] = player_data_df["name"].str.contains(names, case=False, regex=False)
player_data_df = player_data_df.loc[player_data_df["Select"] == True, :]
player_data_df.head()

,id,name,club,age,mkt_value,nationality,Select
0,0,Alexis Sanchez,Arsenal,28,65.0,Chile,True
9,9,Alex Iwobi,Arsenal,21,10.0,Nigeria,True
11,11,Alex Oxlade-Chamberlain,Arsenal,23,22.0,England,True
27,27,Alexandre Lacazette,Arsenal,26,40.0,France,True
231,231,Trent Alexander-Arnold,Liverpool,18,1.5,England,True


In [8]:
# Merge the columns
comb_df = pd.merge(player_data_df, player_info_df, on="name", how="inner")
comb_df
#print(comb_df["player_info"])
#print(comb_df["full_image_url"])

,id,name,club,age,mkt_value,nationality,Select,player_info,full_image_url
0,9,Alex Iwobi,Arsenal,21,10.0,Nigeria,True,https://www.premierleague.com/players/4717/Ale...,https://premierleague-static-files.s3.amazonaw...
1,11,Alex Oxlade-Chamberlain,Arsenal,23,22.0,England,True,https://www.premierleague.com/players/4252/Ale...,https://premierleague-static-files.s3.amazonaw...
2,27,Alexandre Lacazette,Arsenal,26,40.0,France,True,https://www.premierleague.com/players/6899/Ale...,https://premierleague-static-files.s3.amazonaw...
3,231,Trent Alexander-Arnold,Liverpool,18,1.5,England,True,https://www.premierleague.com/players/14732/Tr...,https://premierleague-static-files.s3.amazonaw...


In [9]:
#comb_ls_dict = comb_df.to_dict('records')
comb_dict = comb_df.to_dict('records')
#comb_dict = dict(comb_ls_dict)
comb_dict

[{'id': 9,
  'name': 'Alex Iwobi',
  'club': 'Arsenal',
  'age': 21,
  'mkt_value': 10.0,
  'nationality': 'Nigeria',
  'Select': True,
  'player_info': 'https://www.premierleague.com/players/4717/Alex-Iwobi/overview',
  'full_image_url': 'https://premierleague-static-files.s3.amazonaws.com/premierleague/photos/players/40x40/p153133.png'},
 {'id': 11,
  'name': 'Alex Oxlade-Chamberlain',
  'club': 'Arsenal',
  'age': 23,
  'mkt_value': 22.0,
  'nationality': 'England',
  'Select': True,
  'player_info': 'https://www.premierleague.com/players/4252/Alex-Oxlade-Chamberlain/overview',
  'full_image_url': 'https://premierleague-static-files.s3.amazonaws.com/premierleague/photos/players/40x40/p81880.png'},
 {'id': 27,
  'name': 'Alexandre Lacazette',
  'club': 'Arsenal',
  'age': 26,
  'mkt_value': 40.0,
  'nationality': 'France',
  'Select': True,
  'player_info': 'https://www.premierleague.com/players/6899/Alexandre-Lacazette/overview',
  'full_image_url': 'https://premierleague-static-fil